In [2]:
import os
import torch
from training import train_model, hyperparameter_tuning
from model import AdvancedLSTM

In [3]:
file_path = os.getcwd() + "/data" # run from root
X_train, y_train = torch.load(file_path + "/train_sequences.pt", weights_only=True)
X_val, y_val = torch.load(file_path + "/val_sequences.pt", weights_only=True) 

print(f"Train sequences: {X_train.shape}, Targets: {y_train.shape}")

# Hyperparameter tuning
best_params = hyperparameter_tuning(X_train, y_train, X_val, y_val, epochs=30, n_trials=25)

# Full training with best hyperparameters
print("\nTraining final model with best hyperparameters...")
#model = LSTMEncoderOnly(input_dim=26, hidden_dim=128, output_dim=1, num_layers=2, dropout=0.2)
#model = TransformerEncoderOnly(input_dim=41, hidden_dim=128, output_dim=1, num_layers=2, dropout=0.2, nhead=4)
adv_model = AdvancedLSTM(input_dim=26, hidden_dim=best_params['hidden_dim'], 
                            output_dim=1, num_layers=best_params['num_layers'], 
                            dropout=best_params['dropout'], num_fc_layers=best_params['num_fc_layers'])


train_model(
    adv_model,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val, 
    learning_rate=best_params['learning_rate'],
    weight_decay=best_params['weight_decay'],
    batch_size=best_params['batch_size'],
    weight_ranges=best_params['weight_ranges'],
    epochs=100  # Full training
)

Train sequences: torch.Size([22472, 5, 26]), Targets: torch.Size([22472, 1])
Running random search with 25 trials...

Trial 1/25
Params: {'learning_rate': 0.01, 'hidden_dim': 96, 'weight_decay': 0.0001, 'num_layers': 3, 'dropout': 0.3, 'num_fc_layers': 2, 'batch_size': 256, 'weight_ranges': [(0, 2, 1.0), (2, 4, 1.0), (4, 6, 1.2), (6, 10, 2.0), (10, inf, 3.0)]}


Epoch  1/30:  42%|████▏     | 37/88 [00:04<00:05,  8.69it/s, loss=42.7172, lr=0.000452]


KeyboardInterrupt: 